In [5]:
import pandas as pd
read_file = pd.read_csv (r'C:\Users\admin\Documents\Projects\CS3244-Project\input\Train\X_train.txt')
read_file.to_csv (r'C:\Users\admin\Documents\Projects\CS3244-Project\input\Train\train.csv', index=None, sep=' ')